In [12]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config

import twelvedata
from twelvedata import TDClient


In [13]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [14]:
# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()


In [15]:
# Date range for historical data (funky start date due to downloading first month as test batch)
end_date = datetime.now()
start_date = end_date - timedelta(days=420)  

# Twelve Data API parameters
symbol = "SPX"
interval = "1min"
outputsize = 5000  
parent_df = pd.DataFrame()  # Initialize an empty parent DataFrame

# Increment through the time range in start_date and end_date by 5 days every 30 seconds
while start_date < end_date:
    time_series = td.time_series(
        symbol=symbol,
        interval=interval,
        outputsize=outputsize,
        start_date=start_date.strftime("%Y-%m-%d"),
        end_date=(start_date + timedelta(days=5)).strftime("%Y-%m-%d"),
    ).with_macd().with_ema().with_rsi()
    
    # Convert to pandas Dataframes
    time_series_df = time_series.as_pandas()
    time_series_df = time_series_df.astype(str)
    
    # Concatenate the current time series DataFrame to the parent DataFrame
    parent_df = pd.concat([parent_df, time_series_df], ignore_index=False)
 
    # Increment the start_date
    start_date += timedelta(days=5)
    
    # Sleep for 30 seconds
    time.sleep(6)
    print(start_date)
    

# Print the parent DataFrame to a CSV file
parent_df.to_csv('spx_historical_data.csv', index=True)

2023-06-24 21:38:41.433292
2023-06-29 21:38:41.433292
2023-07-04 21:38:41.433292
2023-07-09 21:38:41.433292
2023-07-14 21:38:41.433292
2023-07-19 21:38:41.433292
2023-07-24 21:38:41.433292
2023-07-29 21:38:41.433292
2023-08-03 21:38:41.433292
2023-08-08 21:38:41.433292
2023-08-13 21:38:41.433292
2023-08-18 21:38:41.433292
2023-08-23 21:38:41.433292
2023-08-28 21:38:41.433292
2023-09-02 21:38:41.433292
2023-09-07 21:38:41.433292
2023-09-12 21:38:41.433292
2023-09-17 21:38:41.433292
2023-09-22 21:38:41.433292
2023-09-27 21:38:41.433292
2023-10-02 21:38:41.433292
2023-10-07 21:38:41.433292
2023-10-12 21:38:41.433292
2023-10-17 21:38:41.433292
2023-10-22 21:38:41.433292
2023-10-27 21:38:41.433292
2023-11-01 21:38:41.433292
2023-11-06 21:38:41.433292
2023-11-11 21:38:41.433292
2023-11-16 21:38:41.433292
2023-11-21 21:38:41.433292
2023-11-26 21:38:41.433292
2023-12-01 21:38:41.433292
2023-12-06 21:38:41.433292
2023-12-11 21:38:41.433292
2023-12-16 21:38:41.433292
2023-12-21 21:38:41.433292
2

In [16]:
print(parent_df)

                           open        high         low       close    volume  \
datetime                                                                        
2023-06-23 15:59:00   4348.1001  4349.87012  4346.47021  4348.22998  10919282   
2023-06-23 15:58:00  4349.18018  4349.81006  4348.22021  4348.27979  21610054   
2023-06-23 15:57:00  4350.29004  4350.33984  4348.77002   4349.1499  21866656   
2023-06-23 15:56:00  4353.83008  4354.16016  4350.20996  4350.20996  22957025   
2023-06-23 15:55:00   4350.6499  4354.14014   4350.6499  4353.68994  23516091   
...                         ...         ...         ...         ...       ...   
2024-08-07 09:34:00   5300.9502  5303.43994  5299.00977   5303.2998  10037694   
2024-08-07 09:33:00  5300.95996  5301.29004  5298.58008  5301.16992  12088593   
2024-08-07 09:32:00   5303.2998  5306.89014  5301.14014  5301.14014  12627988   
2024-08-07 09:31:00  5302.22998   5304.6001   5300.8999  5303.27002  14620887   
2024-08-07 09:30:00  5293.12